# E_weekly_stock_network_visulization
In this temple, we build web based visulatization of weekly dynamic stock networks. See the example in E2_weekly_stock_network_visulization.html

In [7]:
import plotly.graph_objects as go
import networkx as nx
import numpy as np
import pandas as pd
import random
import seaborn as sns
import yfinance as yf
np.random.seed(42)
random.seed(42)

In [8]:
use1 = pd.read_csv("Sym_net_senti.csv").drop(columns = ['Unnamed: 0', 'week_num','mes_count_tt','sym_count','mes_count_symbol','sent_count_sym'])
use1.head()

,user_id,symbol,week,mes_count,user_weight,sym_bull_rate
0,5,AAPL,201831,1,1.000000,0.760773
1,2503,AAPL,201831,5,0.050000,0.760773
2,8154,AAPL,201831,18,0.857143,0.760773
3,8989,AAPL,201831,1,0.045455,0.760773
4,10056,AAPL,201831,8,0.615385,0.760773


In [ ]:
use2 = use1.loc[(use1['user_weight']>=0.2)]

In [23]:
use3 = use2.merge(use2, on = ['user_id','week'],suffixes=['_sym1', '_sym2']).drop(columns = ['sym_bull_rate_sym1','sym_bull_rate_sym2'])
use4 = use3.loc[use3['symbol_sym1'] != use3['symbol_sym2']]
use4['edge_mes'] = use4['mes_count_sym1'] + use4['mes_count_sym2']
use4['edge_weight'] = use4['user_weight_sym1'] + use4['user_weight_sym2']
use5 = use4.drop(columns = ['mes_count_sym1','mes_count_sym2','user_weight_sym1','user_weight_sym2'])

/var/folders/lc/lwj679010v9dq_hdbg2xjrbm0000gn/T/ipykernel_50377/4197114859.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  use4['edge_mes'] = use4['mes_count_sym1'] + use4['mes_count_sym2']
/var/folders/lc/lwj679010v9dq_hdbg2xjrbm0000gn/T/ipykernel_50377/4197114859.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  use4['edge_weight'] = use4['user_weight_sym1'] + use4['user_weight_sym2']


In [25]:
use5.head()

,user_id,symbol_sym1,week,symbol_sym2,edge_mes,edge_weight
5,42934,AAPL,201831,DB,4,0.400000
6,42934,DB,201831,AAPL,4,0.400000
10,75026,AAPL,201831,TSLA,7,1.000000
11,75026,TSLA,201831,AAPL,7,1.000000
18,159063,AAPL,201831,SHOP,67,0.881579


In [28]:
use6 = pd.pivot_table(
    use5,
    index = ['symbol_sym1','symbol_sym2','week'],
    aggfunc = {'user_id':'count', 'edge_mes':np.sum, 'edge_weight':np.sum}
).reset_index()
use6.head()


,symbol_sym1,symbol_sym2,week,edge_mes,edge_weight,user_id
0,A,AB,201933,3,0.600000,1
1,A,AMAT,201920,4,0.444444,1
2,A,AMZN,201936,2,0.400000,1
3,A,AYI,202002,2,0.666667,1
4,A,BABA,201933,3,0.600000,1


In [117]:
node_color = use1[['symbol', 'week','sym_bull_rate']].drop_duplicates()
node_color.head()

,symbol,week,sym_bull_rate
0,AAPL,201831,0.760773
508,AAPL,201911,0.732535
701,SHOP,201911,0.748663
741,AAPL,201912,0.756596
983,AAPL,201926,0.594810


In [113]:
node_color.loc[(node_color['week'] == 201831)&(node_color['symbol'] == 201831)]

,symbol,week,sym_bull_rate
527036,AAOI,201831,0.823529


In [129]:
node_color['sym_bull_rate'].min()

0.0

### Plot One Fig

In [149]:
wi = use6.loc[(use6['week'] == 201911)&(use6['edge_weight'] >= 2)]
lis = []
user_lis = []

for i,d in wi.iterrows():
    user1 = d['symbol_sym1']
    user2 = d['symbol_sym2']
    w = 1 / d['edge_weight']
    c= tuple(sorted([user1,user2]))
    if c not in user_lis:
        user_lis.append(c)
        lis.append((user1,user2,w))
G = nx.Graph((x, y, {'length': v}) for (x, y, v) in sorted(lis))
pos = nx.spring_layout(G, weight = 'length')

In [150]:
edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = pos[edge[0]][0],pos[edge[0]][1]
    x1, y1 = pos[edge[1]][0],pos[edge[1]][1]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=1, color='#888'),
    hoverinfo='text',
    mode='lines')

node_x = []
node_y = []
for node in G.nodes():
    x, y = pos[node][0],pos[node][1]
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    textposition='bottom center',
    mode='markers+text',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))

In [151]:
edge_text = []
for node1,node2,leng in G.edges(data = True):
    edge_text.append(node1 + ' -- ' + node2+ ' with weight '+str(1 / leng['length']))

edge_trace.text = edge_text

In [154]:
node_adjacencies = []
node_text = []
for node, adjacencies in enumerate(G.adjacency()):
    
    l = list(node_color.loc[(node_color['week'] == 201911)&(node_color['symbol'] == adjacencies[0])]['sym_bull_rate'])[0]
    node_adjacencies.append(l)
    node_text.append(adjacencies[0])

node_trace.marker.color = node_adjacencies
node_trace.text = node_text

In [155]:
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='<br>Network graph made with Python',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    #text="Python code: <a href='https://plotly.com/ipython-notebooks/network-graphs/'> https://plotly.com/ipython-notebooks/network-graphs/</a>",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )



fig.show()


### Add slider

In [160]:
def one_week_fig(week, return_scatter = False, node_color = node_color, threshold = 2):
    wi = use6.loc[(use6['week'] == week)&(use6['edge_weight'] >= threshold)]
    lis = []
    user_lis = []
    for i,d in wi.iterrows():
        user1 = d['symbol_sym1']
        user2 = d['symbol_sym2']
        w = 1 / d['edge_weight']
        c= tuple(sorted([user1,user2]))
        if c not in user_lis:
            user_lis.append(c)
            lis.append((user1,user2,w))
    G = nx.Graph((x, y, {'length': v}) for (x, y, v) in sorted(lis))
    pos = nx.spring_layout(G, weight = 'length', seed = 42)
    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = pos[edge[0]][0],pos[edge[0]][1]
        x1, y1 = pos[edge[1]][0],pos[edge[1]][1]
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)

    edge_trace = go.Scatter(
        visible= False,
        x=edge_x, y=edge_y,
        line=dict(width=1, color='#888'),
        hoverinfo='text',
        mode='lines')

    node_x = []
    node_y = []
    for node in G.nodes():
        x, y = pos[node][0],pos[node][1]
        node_x.append(x)
        node_y.append(y)
    node_trace = go.Scatter(
        visible= False,
        x=node_x, y=node_y,
        textposition='bottom center',
        mode='markers+text',
        hoverinfo='text',
        marker=dict(
            showscale=True,
            # colorscale options
            #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
            #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
            #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
            colorscale='YlGnBu',
            reversescale=True,
            color=[],
            size=10,
            colorbar=dict(
                thickness=15,
                title='Sentiment: symbol bullish rate',
                xanchor='left',
                titleside='right'
            ),
            line_width=2))
    node_adjacencies = []
    node_text = []
    for node, adjacencies in enumerate(G.adjacency()):
        l = list(node_color.loc[(node_color['week'] == week)&(node_color['symbol'] == adjacencies[0])]['sym_bull_rate'])[0]
        node_adjacencies.append(l)
        node_text.append(adjacencies[0])

    node_trace.marker.color = node_adjacencies
    node_trace.text = node_text
    if return_scatter:
        return edge_trace, node_trace
    fig = go.Figure(layout=go.Layout(
                title='<br>Network graph made with Python',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    #text="Python code: <a href='https://plotly.com/ipython-notebooks/network-graphs/'> https://plotly.com/ipython-notebooks/network-graphs/</a>",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))
    fig.add_traces([edge_trace, node_trace])
    fig.show()
    return fig

In [162]:
fig = go.Figure()
edge_trace, node_trace = one_week_fig(201931, return_scatter=True)
edge_trace.visible = True
node_trace.visible = True
fig.add_traces([edge_trace, node_trace])
fig.show()

In [165]:
# Create figure
week_lis = sorted(list({i for i in use6['week']}))
fig = go.Figure(layout=go.Layout(
                title='<br>Symbol-symbol Network graph',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    #text="Python code: <a href='https://plotly.com/ipython-notebooks/network-graphs/'> https://plotly.com/ipython-notebooks/network-graphs/</a>",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))
for step in np.arange(0, 100):
    edge_trace, node_trace = one_week_fig(week_lis[step], return_scatter=True)
    fig.add_traces([edge_trace, node_trace])

# Make first week trace visible
fig.data[0].visible = True
fig.data[1].visible = True

# Create and add slider
steps = []
for i in range(0,len(fig.data),2):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Network for week :" + str(week_lis[int(i / 2)])}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    step["args"][0]["visible"][i+1] = True
    steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={"prefix": "Frequency: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders
)

fig.show()

In [166]:
fig.write_html("weekly_stock_network_visulization.html")

In [60]:
type(fig)

plotly.graph_objs._figure.Figure